## インストール

In [ ]:
!pip install -q --no-warn-conflicts "colab-ai-bridge[dspy] @ git+https://github.com/drillan/colab-ai-bridge"

## 使い方

### 基本的な使い方

In [ ]:
from colab_ai_bridge.dspy import ColabDSPyLM
import dspy

# モデルの作成とDSPyの設定（セットアップは自動で完了します）
lm = ColabDSPyLM()
dspy.configure(lm=lm)

# Predictorの実行
predictor = dspy.Predict("question -> answer")
response = predictor(question="フランスの首都は？")
print(response.answer)

### モデルの選択

利用可能なモデルを確認：

In [ ]:
from google.colab import ai

ai.list_models()

特定のモデルを使用：

In [ ]:
from colab_ai_bridge.dspy import ColabDSPyLM
import dspy

# Gemini 2.5 Flash Liteを使用
lm = ColabDSPyLM("google/gemini-2.5-flash-lite")
dspy.configure(lm=lm)

### 構造化出力

DSPy の Signature を使用して型安全な出力を定義：

In [ ]:
from colab_ai_bridge.dspy import ColabDSPyLM
import dspy
from pydantic import BaseModel, Field


class City(BaseModel):
    """都市情報"""

    name: str = Field(description="都市名")
    country: str = Field(description="国名")
    population: int = Field(description="人口")


class CitySignature(dspy.Signature):
    """都市について情報を取得する"""

    query: str = dspy.InputField(desc="都市に関する質問")
    city_info: City = dspy.OutputField(desc="都市情報")


lm = ColabDSPyLM()
dspy.configure(lm=lm)

predictor = dspy.Predict(CitySignature)
response = predictor(query="東京について教えて")
city = response.city_info
print(f"{city.name}, {city.country}, 人口: {city.population:,}")

## サンプル

### サンプル 1: リスト型フィールドを持つ構造化出力

リスト型フィールドを含むデータモデル

In [ ]:
from colab_ai_bridge.dspy import ColabDSPyLM
import dspy
from pydantic import BaseModel, Field


class EmailAddress(BaseModel):
    """メールアドレス情報"""

    email: str = Field(
        pattern=r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$", description="メールアドレス"
    )
    domain: str = Field(description="ドメイン名")


class EmailSignature(dspy.Signature):
    """メールアドレスから情報を抽出する"""

    email_text: str = dspy.InputField(desc="メールアドレスを含むテキスト")
    email_info: EmailAddress = dspy.OutputField(desc="抽出されたメールアドレス情報")


lm = ColabDSPyLM()
dspy.configure(lm=lm)

predictor = dspy.Predict(EmailSignature)
response = predictor(email_text="example@gmail.comのメールアドレス情報を抽出してください")
email = response.email_info
print(f"メール: {email.email}")
print(f"ドメイン: {email.domain}")

### サンプル 2: ChainOfThought による推論

思考の連鎖を使って段階的に推論

In [ ]:
from colab_ai_bridge.dspy import ColabDSPyLM
import dspy
from pydantic import BaseModel, Field


class Book(BaseModel):
    """書籍情報"""

    title: str = Field(description="タイトル")
    author_name: str = Field(description="著者名")
    year: int = Field(description="出版年")
    genres: list[str] = Field(description="ジャンルのリスト")


class BookSignature(dspy.Signature):
    """書籍について詳細な情報を取得する"""

    query: str = dspy.InputField(desc="書籍に関する質問")
    book_info: Book = dspy.OutputField(desc="書籍情報")


lm = ColabDSPyLM()
dspy.configure(lm=lm)

# ChainOfThoughtを使用して段階的に推論
cot = dspy.ChainOfThought(BookSignature)
response = cot(query="村上春樹の『ノルウェイの森』について教えて")
book = response.book_info
print(f"タイトル: {book.title}")
print(f"著者: {book.author_name}")
print(f"出版年: {book.year}")
print(f"ジャンル: {', '.join(book.genres)}")

### サンプル 3: カスタムモジュールの作成

DSPy Module を使ってカスタムロジックを実装

In [ ]:
from colab_ai_bridge.dspy import ColabDSPyLM
import dspy


class HistoricalExplainer(dspy.Module):
    """日本史の専門家として説明するモジュール"""

    def __init__(self):
        super().__init__()
        self.generate = dspy.ChainOfThought("topic -> explanation")

    def forward(self, topic: str) -> dspy.Prediction:
        # システムプロンプトを含めたコンテキストを作成
        context = (
            "あなたは日本史の専門家です。歴史的な事実を正確に、わかりやすく説明してください。\n\n"
            f"トピック: {topic}"
        )
        return self.generate(topic=context)


lm = ColabDSPyLM()
dspy.configure(lm=lm)

explainer = HistoricalExplainer()
result = explainer(topic="関ヶ原の戦いについて簡潔に説明してください")
print(result.explanation)

### サンプル 4: コンテキスト付きプロンプト

ユーザーコンテキストを含むパーソナライズされた応答

In [ ]:
from colab_ai_bridge.dspy import ColabDSPyLM
import dspy
from dataclasses import dataclass


@dataclass
class UserProfile:
    name: str
    age: int
    interests: list[str]


class PersonalizedRecommender(dspy.Module):
    """ユーザープロフィールに基づいた推薦モジュール"""

    def __init__(self):
        super().__init__()
        self.recommend = dspy.Predict("user_info, request -> recommendations")

    def forward(self, user: UserProfile, request: str) -> dspy.Prediction:
        user_info = f"""ユーザー情報:
- 名前: {user.name}
- 年齢: {user.age}
- 興味: {', '.join(user.interests)}"""
        return self.recommend(user_info=user_info, request=request)


lm = ColabDSPyLM()
dspy.configure(lm=lm)

user = UserProfile(name="太郎", age=25, interests=["プログラミング", "機械学習", "読書"])
recommender = PersonalizedRecommender()
result = recommender(user=user, request="おすすめの学習リソースを3つ教えてください")
print(result.recommendations)

### サンプル 5: 温度パラメータによる出力の違い

異なる温度設定での出力比較

In [ ]:
from colab_ai_bridge.dspy import ColabDSPyLM
from colab_ai_bridge.core.config import ModelConfig
import dspy

# 低温度（正確な出力）
config_precise = ModelConfig(temperature=0.3)
lm_precise = ColabDSPyLM(config=config_precise)

# 高温度（創造的な出力）
config_creative = ModelConfig(temperature=1.0)
lm_creative = ColabDSPyLM(config=config_creative)

predictor = dspy.Predict("question -> answer")
prompt = "AIについて一文で説明してください"

print("【正確な出力 (temperature=0.3)】")
dspy.configure(lm=lm_precise)
result1 = predictor(question=prompt)
print(result1.answer)
print()

print("【創造的な出力 (temperature=1.0)】")
dspy.configure(lm=lm_creative)
result2 = predictor(question=prompt)
print(result2.answer)

### サンプル 6: 最適化とキャッシング

DSPy のキャッシュ機能を活用

In [ ]:
from colab_ai_bridge.dspy import ColabDSPyLM
import dspy
import time

# キャッシュ有効（デフォルト）
lm_cached = ColabDSPyLM(cache=True)
dspy.configure(lm=lm_cached)

predictor = dspy.Predict("question -> answer")
question = "日本の四季について説明してください"

print("【1回目（キャッシュなし）】")
start = time.time()
result1 = predictor(question=question)
time1 = time.time() - start
print(f"実行時間: {time1:.2f}秒")
print(result1.answer[:100] + "...")
print()

print("【2回目（キャッシュ使用）】")
start = time.time()
result2 = predictor(question=question)
time2 = time.time() - start
print(f"実行時間: {time2:.2f}秒")
print(result2.answer[:100] + "...")
print()
print(f"速度向上: {time1/time2:.2f}x")